## Import packages

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import hvplot.pandas
from tqdm.notebook import tqdm
from pathlib import Path
from shapely.geometry import box
import holoviews as hv

## Load data

In [ ]:
# define paths
path_lakes = Path('metadata/lakescci_v2.0.2_data-availability.shp')
path_polygons = Path('../../../basin_polygons/shp/basin_polygons.shp')
path_plots = Path('plots/')
path_hylak = Path('E:\RS\HydroLAKES\HydroLAKES_polys_v10_shp\HydroLAKES_polys_v10.shp')

# load data
gdf_basins = gpd.read_file(path_polygons).set_crs(crs='EPSG:4326')
gdf_lakes = gpd.read_file(path_lakes).set_crs(crs='EPSG:4326')

## Generate plots

In [ ]:
import requests
import json
import time
 
def get_lake_name(lat, lon):
    """ Find name of lake at given coords using Overpass API."""
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json][timeout:25];
    is_in({lat}, {lon})->.a;
    nwr(pivot.a)[natural=water];
    out geom;
    """
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    data = response.json()
    if len(data['elements'])==0:
        return None
    else:
        return data['elements'][0]['tags']['name']

In [ ]:
# set bbox buffer (relative)
buffer_factor = 0.1

plotlist = []
for basin in tqdm(gdf_basins.name.unique(), total=gdf_basins.name.unique().shape[0]):
    # load basin
    gdf_basin = gdf_basins.loc[gdf_basins.name==basin]
    
    # get bbox
    minlon, minlat, maxlon, maxlat = gdf_basin.geometry.unary_union.bounds
    buffer_lon = abs(maxlon-minlon)*buffer_factor
    buffer_lat = abs(maxlat-minlat)*buffer_factor
    bbox = (minlon-buffer_lon, minlat-buffer_lat, maxlon+buffer_lon, maxlat+buffer_lat)
    
    # load available lakes
    gdf_lakes['centroid'] = gdf_lakes.geometry.centroid
    gdf_lakes = gdf_lakes.set_geometry('centroid')
    gdf_lakes_inside = gpd.overlay(gdf_lakes, gdf_basin, how='intersection')
    gdf_lakes_outside = gdf_lakes.loc[~gdf_lakes.id.isin(gdf_lakes_inside.id)]
    n_inside = gdf_lakes_inside.shape[0]
    n_outside = gdf_lakes_outside.shape[0]
    
    # load HydroLAKES polygons
    gdf_hylak = gpd.read_file(path_hylak, bbox=bbox).set_crs(crs='EPSG:4326')
    gdf_hylak['Hylak_id_str'] = gdf_hylak['Hylak_id'].astype(str)
    gdf_hylak['Hylak_id'] = gdf_hylak['Hylak_id'].astype(int)
    gdf_hylak = gdf_hylak.set_index('Hylak_id')
    #gdf_hylak_inside = gpd.overlay(gdf_hylak, gdf_basin, how='intersection')
    
    # get hylak information
    df_hylak = pd.DataFrame(gdf_hylak)
    df_hylak_inside = df_hylak.loc[gdf_hylak.sjoin(gdf_lakes_inside).index]
    
    # add lakenames if missing
    for idx, row in df_hylak_inside.iterrows():
        if str(row.Lake_name) == 'nan':
            lon, lat = row.geometry.centroid.coords[0]
            name = get_lake_name(lat, lon)
            df_hylak_inside.loc[idx, 'Lake_name'] = name
    
    # create plot
    plot = gdf_basin.hvplot.polygons(
                title=f'{basin} - Lakes CCI (v2.0.2) (n={n_inside})',
                geo=True, tiles='EsriTerrain',
                label=f'{basin} basin',
                width=800, fontscale=2,
                xlabel='Longitude', ylabel='Latitude',
                line_alpha=0,
                color='red', fill_alpha=0.2, legend='bottom_left', 
                xlim=(bbox[0], bbox[2]), ylim=(bbox[1], bbox[3])) * \
            gdf_hylak.hvplot.polygons(
                geo=True, features={'rivers':'10m'}, color='#89CFF0', line_color=None, label='Lakes') * \
            gdf_lakes_inside.hvplot.points(
                geo=True, color='#109618', size=50, line_color='black', 
                label='Lake (inside)')* \
            gdf_lakes_outside.hvplot.points(
                geo=True, color='#DC3912', size=50, line_color='black', 
                label='Lake (outside)')
    
    plot_table = df_hylak_inside.hvplot.table(
                columns=['Hylak_id_str', 'Lake_name', 'Lake_area', 'Country', 'Pour_long', 'Pour_lat'],
                selectable=True, sortable=True, width=800)
    plot_combined = hv.Layout([plot, plot_table]).cols(1)
    
    plotlist.append(plot_combined)
    
    # Export plot
    hv.save(plot_combined, path_plots.joinpath(f'{basin}_ccilakes.png'), fmt='png', dpi=300)
    hv.save(plot_combined, path_plots.joinpath(f'{basin}_ccilakes.html'), fmt='html')